## Import sections and convert

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from shapely.geometry import shape, Polygon, LineString, Point
from shapely.ops import nearest_points
from math import acos, degrees, sqrt
from map2loop import m2l_utils
%matplotlib inline


In [ ]:
seismic_line_file='../test_data3/data/seismic_line_10GA-CP1_rev.shp'   #input geology file (if local)

seismic_line = gpd.read_file(seismic_line_file) #import map

seismic_line.plot(figsize=(10,10),edgecolor='#000000',linewidth=0.2) #display map
display(seismic_line)


In [ ]:
seismic_bbox_file='../test_data3/data/seismic_bbox.shp'   #input geology file (if local)

seismic_bbox = gpd.read_file(seismic_bbox_file) #import map

seismic_bbox.plot(figsize=(10,10),edgecolor='#000000',linewidth=0.2) #display map
seismic_bbox.set_index('POSITION',inplace=True)
display(seismic_bbox,seismic_bbox.loc['BR'])

In [ ]:
seismic_interp_file='../test_data3/data/seismic_interp.shp'   #input geology file (if local)

seismic_interp = gpd.read_file(seismic_interp_file) #import map

seismic_interp.plot(column='FEATURE',figsize=(10,10),edgecolor='#000000',linewidth=0.5) #display map
display(seismic_interp)

In [ ]:
dtm_reproj_file='../test_data3/dtm/dtm_rp.tif'
dtm = rasterio.open(dtm_reproj_file)


def to convert section x,y to real_world x,y,z based on dtm & seismic line m2l_section.section2model()
parse section geodataframe and save fault location info below buffer depth m2l_section.process_faults()
parse section geodataframe and save basal contact location info below buffer depth m2l_section.process_strat()
parse section geodataframe and process fault-fault topology and fault-strat topology and strat topology m2l_section.process_topology()



In [ ]:
def bboxscale(seismic_bbox):
    xscale=seismic_bbox.loc['TR'].geometry.x-seismic_bbox.loc['TL'].geometry.x
    yscale=seismic_bbox.loc['TR'].geometry.y-seismic_bbox.loc['BR'].geometry.y
    return(xscale,yscale)

xscale,yscale=bboxscale(seismic_bbox)

print(xscale,yscale)

In [ ]:
def section2model(seismic_line,seismic_bbox,sx,sy):
    sx1=(sx-seismic_bbox.loc['TL'].geometry.x)/(seismic_bbox.loc['TR'].geometry.x-seismic_bbox.loc['TL'].geometry.x)
    sy1=(sy-seismic_bbox.loc['TR'].geometry.y)
    for indx,lines in seismic_line.iterrows():
        s_ls=LineString(lines.geometry)
        full_dist=s_ls.length
        break
        
    for indx,lines in seismic_line.iterrows():
        s_ls=LineString(lines.geometry)
        first=True
        cdist=0
        for seg in s_ls.coords:        
            if(not first):
                dist=m2l_utils.ptsdist(seg[0],seg[1],lsegx,lsegy)
                cdist=cdist+dist
                norm_dist=cdist/full_dist
                if(sx1>last_norm_dist and sx1<norm_dist):
                    local_norm=((sx1-last_norm_dist)/(norm_dist-last_norm_dist))
                    mx=lsegx+((seg[0]-lsegx)*local_norm)
                    my=lsegy+((seg[1]-lsegy)*local_norm)
                    return(mx,my)
                lsegx=seg[0]
                lsegy=seg[1]
                last_norm_dist=norm_dist
            else:
                first=False
                lsegx=seg[0]
                lsegy=seg[1]
                last_norm_dist=0
        return(-999,-999)


In [ ]:
sx,sy=1730.227349563404, -751.33526691666

mx,my=section2model(seismic_line,seismic_bbox,sx,sy)
print(mx,my)

In [ ]:
def extract_section(tmp_path,output_path,seismic_line,seismic_bbox,seismic_interp,dtm,surface_cut):
    fault_clip_file=tmp_path+'faults_clip.shp'   
    faults = gpd.read_file(fault_clip_file) #import faults    
    all_sorts=pd.read_csv(tmp_path+'all_sorts2.csv',",")
    sf=open(output_path+'seismic_faults.csv',"w")
    sf.write('X,Y,Z,formation\n')
    sb=open(output_path+'seismic_base.csv',"w")
    sb.write('X,Y,Z,formation\n')
    for indx,interps in seismic_interp.iterrows():
        i_ls=LineString(interps.geometry)
        for seg in i_ls.coords:
            mx,my=section2model(seismic_line,seismic_bbox,seg[0],seg[1])
            if( mx != -999 and  my != -999):
                mz=seismic_bbox.loc['BR']['DEPTH']*(seismic_bbox.loc['TR'].geometry.y-seg[1])/(seismic_bbox.loc['TR'].geometry.y-seismic_bbox.loc['BR'].geometry.y)
                locations=[(mx,my)]
                height= m2l_utils.value_from_raster(dtm,locations)
                if(not height==-999 and mz>surface_cut):
                    mz2=-mz+float(height)
                    #print(mx,my,mz,height,mz2)
                    if(str(interps['IDENT'])=='None'):
                        ident='None'
                    else:
                        ident=str(interps['IDENT'])
                    if('Base' in interps['FEATURE']):
                        maxfm=0
                        maxname=''
                        for indx,formation in all_sorts.iterrows():
                            if(formation['group'] in interps['IDENT'] and formation['index in group']>maxfm):
                                maxfm=formation['index in group']
                                maxname=formation['code']
                        ostr=str(mx)+','+str(my)+','+str(mz2)+','+maxname+'\n'
                        sb.write(ostr)
                    else:
                        for indx,aflt in faults.iterrows():
                            if(not str(aflt['NAME'])=='None' and not ident == 'None'):
                                fname=aflt['NAME'].replace(" ","_")
                                if(fname in interps['IDENT'] ):
                                    fault_id='Fault_'+str(aflt['OBJECTID'])
                                    ostr=str(mx)+','+str(my)+','+str(mz2)+','+fault_id+'\n'
                                    sf.write(ostr)
                                    break
    sf.close()
    sb.close()


In [ ]:
tmp_path='../test_data3/tmp/'
output_path='../test_data3/output/'

extract_section(tmp_path,output_path,seismic_line,seismic_bbox,seismic_interp,dtm,2000)

In [ ]:
all_sorts=pd.read_csv('../test_data3/tmp/all_sorts2.csv',",")
display(all_sorts)

In [ ]:
maxfm=0
maxname=''
for indx,formation in all_sorts.iterrows():
    if(formation['group'] in 'Fortescue_Group' and formation['index in group']>maxfm):
        maxfm=formation['index in group']
        maxname=formation['code']
print('base fm =',maxfm,maxname)

In [ ]:
fault_clip_file='../test_data3/tmp/faults_clip.shp'   

faults = gpd.read_file(fault_clip_file) #import faults
display(faults)

In [ ]:
for indx,aflt in faults.iterrows():
    if(not str(aflt['NAME'])=='None'):
        fname=aflt['NAME'].replace(" ","_")
        if(fname in 'Coppin_Fault' ):
            fault_id='Fault_'+str(aflt['OBJECTID'])
            print(fname,fault_id)
            break

In [ ]:
fault_orientation_file=output_path+'fault_orientations.csv'
fault_clip_file=tmp_path+'faults_clip.shp'

def assign_fault_dips(seismic_interp_file,fault_clip_file,fault_orientation_file,seismic_bbox):
    fault_orientations=pd.read_csv(fault_orientation_file,",")
    fault_orientations.set_index('formation',inplace=True)
    #display(fault_orientations)
    faults = gpd.read_file(fault_clip_file) #import faults    
    interps = gpd.read_file(seismic_interp_file) #import interps
    
    for ind,interps in interps.iterrows():
        if('Fault' in interps['FEATURE'] and 'None' in str(interps['IDENT'])):
            #display(interps)
            i_ls=LineString(interps.geometry)
            pts=len(interps.geometry.coords)
            if(interps.geometry.coords[0][1]<interps.geometry.coords[pts-1][1]):
                shallowestx=interps.geometry.coords[pts-1][0]
                shallowesty=interps.geometry.coords[pts-1][1]
                nshallowestx=interps.geometry.coords[pts-2][0]
                nshallowesty=interps.geometry.coords[pts-2][1]
            else:
                shallowestx=interps.geometry.coords[0][0]
                shallowesty=interps.geometry.coords[0][1]
                nshallowestx=interps.geometry.coords[1][0]
                nshallowesty=interps.geometry.coords[1][1]
            print(ind)    
            smx,smy=section2model(seismic_line,seismic_bbox,shallowestx,shallowesty)
            if( smx != -999 and smy != -999):
                nsmx,nsmy=section2model(seismic_line,seismic_bbox,nshallowestx,nshallowesty)
                smz=-seismic_bbox.loc['BR']['DEPTH']*(seismic_bbox.loc['TR'].geometry.y-shallowesty)/(seismic_bbox.loc['TR'].geometry.y-seismic_bbox.loc['BR'].geometry.y)
                nsmz=-seismic_bbox.loc['BR']['DEPTH']*(seismic_bbox.loc['TR'].geometry.y-nshallowesty)/(seismic_bbox.loc['TR'].geometry.y-seismic_bbox.loc['BR'].geometry.y)
                #print(shallowestx,shallowesty,nshallowestx,nshallowesty,smx,smy,smz,nsmx,nsmy,nsmz)

                dx=sqrt(((smx-nsmx)*(smx-nsmx))+((smy-nsmy)*(smy-nsmy)))
                dy=smz-nsmz
                fl,fm=m2l_utils.pts2dircos(0,0,dx,dy)
                dip=-(90-degrees(acos(fm)))
                #print('dip',fl,fm,-dip)

                closest=1e9
                close_indx=-1
                ipt=Point((smx,smy))
                for indx,aflt in faults.iterrows():
                    close=nearest_points(aflt.geometry, ipt)
                    #print(close[0].x,close[0].y,close[1].x,close[1].y)
                    dist=m2l_utils.ptsdist(close[0].x,close[0].y,close[1].x,close[1].y)
                    if( 'Fault_'+str(aflt['OBJECTID']) in fault_orientations.index):
                        fid='Fault_'+str(aflt['OBJECTID'])
                        ddir=fault_orientations.loc[fid]['DipDirection']
                        tracel,tracem=m2l_utils.pts2dircos(smx,smy,nsmx,nsmy)
                        fl,fm,fn=m2l_utils.ddd2dircos(90.0,ddir)
                        angle=degrees(acos((tracel*fm)+(-tracem*fl))) #swap because dip direction
                        #print(fid,angle,tracel,tracem,fl,fm)
                        if(dist<closest and dist < 10000 and angle>40 and angle < 140 ):
                            closest=dist
                            close_indx=indx
                            angle_max=angle
                            ddirmax=ddir
                            fidmax=fid
                if(angle > 90):
                    dipdir=ddirmax
                else:
                    dipdir=ddirmax+180
                #print(fidmax,smx,smy,close_indx,closest,faults.iloc[close_indx]['OBJECTID'],angle,ddirmax,dipdir)
                print(fidmax,dipdir,dip)
            
            # figure out if correct yup
            # understand >90 values gone
            # loop thorugh fault_orientations to find closest fault:
            #          assign fault dip to nearest fault that CUTS line at HIGH ANGLE!!! 
            #          but local angle hard to estimate from global
            # loop through fault_orientations:
            #          write out new fault_orientations file with updated orientation 
            # NOT WORKING #########   (with test of dip directon wrt to fault string ordering)
            # basically matching surface ruptures to seismic faults is really hard, even when they are both named!

assign_fault_dips(seismic_interp_file,fault_clip_file,fault_orientation_file,seismic_bbox)            